In [4]:
using Revise
#using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
#using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [5]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [6]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [7]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi, dy)

291×5 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬────────────────────┬─────────────┬─────────────────┬─────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI  │ Yeld_10Y_T_Bill │ CPI        ⋯
├────────────┼────────────────────┼─────────────┼─────────────────┼─────────────
│ 1952-03-01 │        -0.00772801 │  0.00772445 │       0.0022875 │ -0.0037807 ⋯
│ 1952-06-01 │          0.0651996 │ -0.00708528 │       0.0017875 │   0.011299 ⋯
│ 1952-09-01 │         -0.0227685 │  0.00423886 │       0.0024375 │         0. ⋯
│ 1952-12-01 │            0.08432 │   0.0089235 │       0.0011425 │ -0.0037523 ⋯
│ 1953-03-01 │         -0.0487217 │  0.00497517 │       0.0018125 │         0. ⋯
│ 1953-06-01 │        -0.00778441 │ -0.00227055 │       0.0021925 │  0.0074906 ⋯
│ 1953-09-01 │        -0.00277599 │ -0.00299956 │        0.002115 │  0.0074349 ⋯
│ 1953-12-01 │          0.0705234 │  0.00767892 │       0.0021725 │ -0.0037105 ⋯
│     ⋮      │         ⋮         

In [8]:
CSV.write("usa_data_transformed.csv", data)

UndefVarError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [82]:
start = Date(1975,01,01)
end_d = Date(2020, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1395,0.8255,0.1138,0.2722,-1.0147,-0.6321,-0.0168,0.0909,1.6908,0.8155,0.1398


In [83]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.0701,0.1868,0.0175,0.0623,-0.2565,-0.5872,-0.0166,0.0868,1.3255,0.2501,0.0218
T_Bill_CPI,0.0083,0.0255,0.0019,0.0073,-0.0208,0.0704,-0.0038,0.3505,0.2942,0.2237,0.0024
Yeld_10Y_T_Bill,0.0031,0.0099,0.0043,0.0066,-0.0077,-0.7037,0.0021,0.769,0.7198,1.3822,0.0048
CPI,0.009,0.0403,0.0088,0.0124,-0.0254,0.3024,0.004,0.4101,2.0781,0.9769,0.0082
Dividend_Yeld,0.4315,-2.7747,-3.6729,-3.3352,-4.4867,0.2391,-4.0007,0.9781,-1.0182,-8.5116,-3.8214


In [84]:
returns_summarystats(s_data, freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1402,0.7471,0.0702,0.2492,-1.026,-0.5872,-0.0662,0.0868,1.3255,0.5003,0.0873
T_Bill_CPI,0.0166,0.1019,0.0074,0.0291,-0.0832,0.0704,-0.0152,0.3505,0.2942,0.4474,0.0095
Yeld_10Y_T_Bill,0.0062,0.0397,0.0172,0.0264,-0.0306,-0.7037,0.0084,0.769,0.7198,2.7644,0.019
CPI,0.018,0.1612,0.0351,0.0496,-0.1016,0.3024,0.0159,0.4101,2.0781,1.9539,0.0326
Dividend_Yeld,0.863,-11.0987,-14.6915,-13.3408,-17.9466,0.2391,-16.0028,0.9781,-1.0182,-17.0231,-15.2855


In [85]:
s_data_cor = cor(values(s_data))
pretty_table(s_data_cor, backend = Val(:html),header=colnames(s_data), row_labels=colnames(s_data), title="correlatio matrix")

,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld
Total_Stock_T_Bill,1.0,-0.0600267,0.0491574,-0.0402203,-0.0897381
T_Bill_CPI,-0.0600267,1.0,-0.0795142,-0.530145,0.189184
Yeld_10Y_T_Bill,0.0491574,-0.0795142,1.0,-0.258195,0.0791524
CPI,-0.0402203,-0.530145,-0.258195,1.0,0.464042
Dividend_Yeld,-0.0897381,0.189184,0.0791524,0.464042,1.0


In [86]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld,std
Total_Stock_T_Bill,0.2481,0.0922,-2.1897,-3.4561,-3.1543,0.0507,0.0684
T_Bill_CPI,-0.0344,-0.0119,0.855,0.7645,0.7257,-0.0069,0.007
Yeld_10Y_T_Bill,0.0017,-0.0036,-0.002,0.7703,-0.0041,0.0002,0.002
CPI,0.0361,0.02,0.1018,-0.6647,0.2425,0.0074,0.0074
Dividend_Yeld,-0.1911,-0.0795,0.927,2.5259,2.2616,0.958,0.0713


,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld
Total_Stock_T_Bill,1.0,0.03,0.08,-0.08,-0.97
T_Bill_CPI,0.03,1.0,-0.14,-0.95,-0.05
Yeld_10Y_T_Bill,0.08,-0.14,1.0,-0.11,-0.08
CPI,-0.08,-0.95,-0.11,1.0,0.09
Dividend_Yeld,-0.97,-0.05,-0.08,0.09,1.0


In [87]:
isposdef(vmodel.Σ)

false

In [72]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

PosDefException: PosDefException: matrix is not Hermitian; Factorization failed.

In [74]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

PosDefException: PosDefException: matrix is not Hermitian; Factorization failed.

In [66]:
y_data = annualise(scenarios[1, :, 21:25],4)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [39]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

MethodError: MethodError: no method matching size(::Missing)
The function `size` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  size(!Matched::Base.MethodList)
   @ Base reflection.jl:1193
  size(!Matched::Core.Compiler.StmtRange)
   @ Base show.jl:2820
  size(!Matched::XLSX.CellRange)
   @ XLSX C:\Users\matsz\.julia\packages\XLSX\vzApD\src\cellref.jl:200
  ...


In [40]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [41]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [42]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

KeyError: KeyError: key :Price_10Y_T_Bill not found

In [43]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



UndefVarError: UndefVarError: `mu` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [44]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


In [45]:
returns_summarystats(data_source[:Yeld_10Y],1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0292,0.1584,0.0548,0.0736,0.0066,0.9377,0.0333,0.9774,0.6945,1.8772,0.0468


In [46]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [47]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [48]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [49]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [50]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18202307073348617
 -0.02765639847268144
  0.07979295713853621
  0.18722984282522268
  0.3425419454786437

In [51]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18362104611967509
 -0.02647412363134339
  0.08104147818774934
  0.18828392285982776
  0.3407729685167871

In [52]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

5-element Vector{Float64}:
 -0.25965526319176224
 -0.018917595580195047
  0.03984357221170979
  0.09899454850125108
  0.33237652565958525

In [53]:
r = rand(Normal(0,3),10_000)
display(skewness(r))
display(kurtosis(r))

-0.005607860428916721

-0.01752304277342054